In [1]:
import scanpy as sc
import squidpy as sq

import numpy as np
import pandas as pd

In [2]:
adata1 = sc.read_h5ad('datasets/merfish/H18.06.006.MTG.4000.expand.rep1.h5ad')
adata2 = sc.read_h5ad('datasets/merfish/H18.06.006.MTG.4000.expand.rep2.h5ad')
adata3 = sc.read_h5ad('datasets/merfish/H18.06.006.MTG.4000.expand.rep3.h5ad')
adata1.obs['batch'] = 'H18.06.006.MTG.4000.rep1'
adata2.obs['batch'] = 'H18.06.006.MTG.4000.rep2'
adata3.obs['batch'] = 'H18.06.006.MTG.4000.rep3'

adata1.obs['coord'] = adata1.obsm['spatial'][:,1]
adata1.obs['niche'] = 'N1'
adata1.obs['niche'][adata1.obs['coord']> -1500] = 'N2'
adata1.obs['niche'][adata1.obs['coord']> -1000] = 'N3'
adata1.obs['niche'][adata1.obs['coord']> -200] = 'N4'
adata1.obs['niche'][adata1.obs['coord']> 350] = 'N5'

adata2.obs['coord'] = np.sqrt((adata2.obsm['spatial'][:,0]+3000) ** 2 + (adata2.obsm['spatial'][:,1]-400) ** 2)
adata2.obs['niche'] = 'N6'
adata2.obs['niche'][adata2.obs['coord']> 2100] = 'N5'
adata2.obs['niche'][adata2.obs['coord']> 3600] = 'N4'

adata3.obs['coord'] = np.sqrt((adata3.obsm['spatial'][:,0]-3000) ** 2 + (adata3.obsm['spatial'][:,1]-1300) ** 2)
adata3.obs['niche'] = 'N6'
adata3.obs['niche'][adata3.obs['coord']> 2000] = 'N5'
adata3.obs['niche'][adata3.obs['coord']> 3900] = 'N4'
adata3.obs['niche'][adata3.obs['coord']> 4350] = 'N3'
adata3.obs['niche'][adata3.obs['coord']> 5600] = 'N2'
adata3.obs['niche'][adata3.obs['coord']> 6000] = 'N1'

adata_mtg = sc.concat([adata1,adata2,adata3])
adata_mtg.obs_names_make_unique()
adata_mtg

/tmp/ipykernel_4091933/3933570242.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata1.obs['niche'][adata1.obs['coord']> -1500] = 'N2'
/tmp/ipykernel_4091933/3933570242.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata1.obs['niche'][adata1.obs['coord']> -1000] = 'N3'
/tmp/ipykernel_4091933/3933570242.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata1.obs['niche'][adata1.obs['coord']> -200] = 'N4'
/tmp/ipykernel

AnnData object with n_obs × n_vars = 11059 × 4000
    obs: 'name', 'global.x', 'global.y', 'adjusted.x', 'adjusted.y', 'fov.x', 'fov.y', 'cluster_L1', 'cluster_L2', 'cluster_L3', 'batch', 'coord', 'niche'
    obsm: 'spatial'

In [3]:
# graphonly models

from simvi.model import SimVIGraphOnly

Global seed set to 0


In [28]:
SimVIGraphOnly.setup_anndata(adata_mtg,batch_key='batch')
edge_index = SimVIGraphOnly.extract_edge_index(adata_mtg,n_neighbors=10,batch_key='batch')

## likelihood

from pytorch_lightning.utilities.seed import seed_everything
for i in range(10):
    seed_everything(i)
    model = SimVIGraphOnly(adata_mtg,kl_weight=1,kl_gatweight=1,lam_mi=500,permutation_rate=0,n_spatial=20)
    train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=500,use_gpu=True)
    adata_mtg.obsm['simvigo_s_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)

Global seed set to 0
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:35<00:00,  2.82it/s, train_loss=1.47e+3, val_loss=1541.2582]
Global seed set to 1
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:34<00:00,  2.92it/s, train_loss=1.47e+3, val_loss=1538.1469]
Global seed set to 2
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:34<00:00,  2.86it/s, train_loss=1.47e+3, val_loss=1546.6969]
Global seed set to 3
Epoch 100/100: 100%|███████████████████████████████████████████████████████████████████████████████████

In [29]:
adata_mtg.obs['batch_ct'] = adata_mtg.obs['batch'].astype(str) + adata_mtg.obs['cluster_L1'].astype(str)

SimVIGraphOnly.setup_anndata(adata_mtg,batch_key='batch_ct')
edge_index = SimVIGraphOnly.extract_edge_index(adata_mtg,n_neighbors=10,batch_key='batch')

from pytorch_lightning.utilities.seed import seed_everything
for i in range(10):
    seed_everything(i)
    model = SimVIGraphOnly(adata_mtg,kl_weight=1,kl_gatweight=1,lam_mi=500,permutation_rate=0,n_spatial=20)
    train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=500,use_gpu=True)
    adata_mtg.obsm['simvigoct_s_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)

Global seed set to 0
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:35<00:00,  2.86it/s, train_loss=1.44e+3, val_loss=1452.544]
Global seed set to 1
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:35<00:00,  2.81it/s, train_loss=1.44e+3, val_loss=1447.3746]
Global seed set to 2
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:34<00:00,  2.87it/s, train_loss=1.44e+3, val_loss=1448.0414]
Global seed set to 3
Epoch 100/100: 100%|███████████████████████████████████████████████████████████████████████████████████

In [30]:
adata_mtg.obs['batch_ct2'] = adata_mtg.obs['batch'].astype(str) + adata_mtg.obs['cluster_L2'].astype(str)

SimVIGraphOnly.setup_anndata(adata_mtg,batch_key='batch_ct2')
edge_index = SimVIGraphOnly.extract_edge_index(adata_mtg,n_neighbors=10,batch_key='batch')

from pytorch_lightning.utilities.seed import seed_everything
for i in range(10):
    seed_everything(i)
    model = SimVIGraphOnly(adata_mtg,kl_weight=1,kl_gatweight=1,lam_mi=500,permutation_rate=0,n_spatial=20)
    train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=500,use_gpu=True)
    adata_mtg.obsm['simvigoct2_s_'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)

/usr/local/lib/python3.8/dist-packages/scvi/data/_utils.py:156: UserWarning: Category 39 in adata.obs['_scvi_batch'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
Global seed set to 0
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:34<00:00,  2.87it/s, train_loss=1.43e+3, val_loss=1440.771]
Global seed set to 1
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:35<00:00,  2.85it/s, train_loss=1.43e+3, val_loss=1440.337]
Global seed set to 2
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 

In [14]:
import scvi
scvi.model.SCVI.setup_anndata(adata_mtg,batch_key='batch')
for i in range(10):
    seed_everything(i)
    vae = scvi.model.SCVI(adata_mtg, n_layers=2, n_latent=20)
    vae.train(max_epochs=100,use_gpu=True,batch_size=500)
    adata_mtg.obsm['scvi_'+str(i)] = vae.get_latent_representation(adata_mtg)

Global seed set to 0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:31<00:00,  3.15it/s, loss=1.42e+03, v_num=1]

Global seed set to 1


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:31<00:00,  3.18it/s, loss=1.42e+03, v_num=1]

...

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:31<00:00,  3.14it/s, loss=1.42e+03, v_num=1]

Global seed set to 9


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:31<00:00,  3.17it/s, loss=1.42e+03, v_num=1]


In [3]:
from simvi.model import SimVI
SimVI.setup_anndata(adata_mtg,batch_key='batch')
edge_index = SimVI.extract_edge_index(adata_mtg,n_neighbors=10,batch_key='batch')
from pytorch_lightning.utilities.seed import seed_everything
for i in range(10):
    seed_everything(i)
    model = SimVI(adata_mtg,kl_weight=1,kl_gatweight=0.01,lam_mi=1000,permutation_rate=0.5,n_spatial=20,n_intrinsic=20)
    train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=500,use_gpu=True,mae_epochs=25)
    adata_mtg.obsm['simvikl_z'+str(i)] = model.get_latent_representation(edge_index,representation_kind='intrinsic',give_mean=True)
    adata_mtg.obsm['simvikl_lz'+str(i)] = model.get_latent_representation(edge_index,representation_kind='output',give_mean=True)
    adata_mtg.obsm['simvikl_s'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)

Global seed set to 0
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:44<00:00,  2.24s/it, train_loss=1.43e+3, val_loss=1435.4552]
Global seed set to 1
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:49<00:00,  2.29s/it, train_loss=1.43e+3, val_loss=1436.5302]
Global seed set to 2
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:48<00:00,  2.29s/it, train_loss=1.43e+3, val_loss=1433.8075]
Global seed set to 3
Epoch 100/100: 100%|███████████████████████████████████████████████████████████████████████████████████

In [7]:
adata_mtg.write('mtg_baselines2.h5ad')